In [0]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

In [0]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')



In [0]:
# Hyper parameters
num_epochs = 300
num_classes = 10
batch_size = 64
learning_rate = 0.001

In [0]:
normalize = transforms.Normalize(mean=[.5, .5, .5],
                                 std=[1 ,1, 1])
transform_ = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

In [5]:
# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform_,
                                             download=True)

0it [00:00, ?it/s]

170500096it [00:06, 28082388.76it/s]                               


Extracting ../../data/cifar-10-python.tar.gz to ../../data/


In [6]:
image, label = train_dataset[0]
print (image.size())
print (label)


torch.Size([3, 32, 32])
6


In [0]:
# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,   
                                           batch_size=batch_size, 
                                           shuffle=True,
                                          )

In [0]:
# data_iter = iter(train_loader)
# images, labels = data_iter.next()


In [0]:
# images.max()

In [0]:
test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


In [0]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 24, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(24, 48, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(48, 96, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(96),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Sequential(
            nn.Linear(2*2*96, 1024),
            nn.ReLU())
        self.fc2 = nn.Linear(1024, num_classes)
        self.sm = nn.Softmax()
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.sm(out)
        return out

In [0]:
model = ConvNet(num_classes).to(device)


In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) #if needed momentum=0.9


In [0]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [1/300], Step [100/782], Loss: 2.3025
Epoch [1/300], Step [200/782], Loss: 2.2984
Epoch [1/300], Step [300/782], Loss: 2.3002
Epoch [1/300], Step [400/782], Loss: 2.2974
Epoch [1/300], Step [500/782], Loss: 2.2957
Epoch [1/300], Step [600/782], Loss: 2.2942
Epoch [1/300], Step [700/782], Loss: 2.2980
Epoch [2/300], Step [100/782], Loss: 2.2915
Epoch [2/300], Step [200/782], Loss: 2.2900
Epoch [2/300], Step [300/782], Loss: 2.2811
Epoch [2/300], Step [400/782], Loss: 2.2805
Epoch [2/300], Step [500/782], Loss: 2.2793
Epoch [2/300], Step [600/782], Loss: 2.2878
Epoch [2/300], Step [700/782], Loss: 2.2754
Epoch [3/300], Step [100/782], Loss: 2.2887
Epoch [3/300], Step [200/782], Loss: 2.2757
Epoch [3/300], Step [300/782], Loss: 2.2592
Epoch [3/300], Step [400/782], Loss: 2.2707
Epoch [3/300], Step [500/782], Loss: 2.2553
Epoch [3/300], Step [600/782], Loss: 2.2450
Epoch [3/300], Step [700/782], Loss: 2.2474
Epoch [4/300], Step [100/782], Loss: 2.2684
Epoch [4/300], Step [200/782], L

In [0]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')